In [502]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder,normalize
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,ConfusionMatrixDisplay
from sklearn import feature_selection
from scipy.stats import chi2_contingency
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,precision_recall_curve
from sklearn.feature_selection import SelectFromModel
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import pickle as pk
import numpy as np

In [503]:
# %pip install --user xgboost

In [504]:
# %conda install -c conda-forge py-xgboost-gpu

In [505]:
data_prof = pd.read_csv('Data_Arbre.csv')
data_debase = pd.read_csv('Patrimoine_Arbore.csv', encoding='latin1')

In [506]:
cols = ["fk_arb_etat","tronc_diam","haut_tot","fk_stadedev","age_estim", "clc_quartier", "feuillage", "fk_port", "fk_nomtech", "fk_revetement"]
data = data_prof[cols]

index = data[(data["fk_arb_etat"] == "Essouché") | (data["fk_arb_etat"] == "Non essouché")]
index = data[(data["fk_arb_etat"] == "SUPPRIMÉ") | 
             (data["fk_arb_etat"] == "ABATTU") | 
             (data["fk_arb_etat"] == "EN PLACE") | 
             (data["fk_arb_etat"] == "REMPLACÉ")].index

data.drop(index, inplace=True)

data.loc[data["fk_arb_etat"] == "Essouché", "fk_arb_etat"] = 1
data.loc[data["fk_arb_etat"] == "Non essouché", "fk_arb_etat"] = 0

# data.loc[data["fk_arb_etat"] != 1, "fk_arb_etat"] = 0
data.fk_arb_etat = data.fk_arb_etat.astype(int)
# data.fk_arb_etat.value_counts()

C:\Users\Alexandre\AppData\Local\Temp\ipykernel_10236\1887067551.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(index, inplace=True)
C:\Users\Alexandre\AppData\Local\Temp\ipykernel_10236\1887067551.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.fk_arb_etat = data.fk_arb_etat.astype(int)


In [507]:
#Encodage
#Get dummies de pandas encoder en hot one
encoder = OrdinalEncoder()
lst_col = ['fk_stadedev', 'clc_quartier', 'feuillage', 'fk_port', 'fk_nomtech', "fk_revetement"]

temp = data[lst_col]

data[lst_col] = encoder.fit_transform(temp)

C:\Users\Alexandre\AppData\Local\Temp\ipykernel_10236\992099324.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[lst_col] = encoder.fit_transform(temp)


In [508]:
X_data = data.drop(columns=["fk_arb_etat"])
Y_data = data["fk_arb_etat"]

# X_data = normalize(X_data)


X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42, stratify=Y_data)

In [509]:
# Y_test.value_counts()

MLP ?

In [510]:
from sklearn.neural_network import MLPClassifier

In [511]:
mlp = MLPClassifier(hidden_layer_sizes=(20, 50),validation_fraction=0.2, learning_rate='adaptive')
mlp.fit(X_train, Y_train)

c:\Users\Alexandre\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(20, 50), learning_rate='adaptive',
              validation_fraction=0.2)

In [512]:
pred = mlp.predict(X_train)
# mlp.score(X_test, Y_test)
# pred
score = accuracy_score(Y_train, pred)
score

0.8954248366013072

In [513]:
pred = mlp.predict(X_train)
mat  = confusion_matrix(Y_train, pred, normalize='true')
mat

array([[0.31818182, 0.68181818],
       [0.00763359, 0.99236641]])

In [514]:
pred2 = mlp.predict(X_test)
mat  = confusion_matrix(Y_test, pred2, normalize='true')
mat

array([[0.2       , 0.8       ],
       [0.05882353, 0.94117647]])

SMOTE

In [515]:
# sm = SMOTE(random_state=42, n_jobs=-1, sampling_strategy=0.4)
sm = SMOTE(random_state=42, n_jobs=-1)
X_smote, Y_smote = sm.fit_resample(X_train, Y_train)

c:\Users\Alexandre\anaconda3\Lib\site-packages\imblearn\over_sampling\_smote\base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


In [516]:
mlp_smoted = MLPClassifier(hidden_layer_sizes=(20, 50),validation_fraction=0.2, learning_rate='adaptive')
mlp_smoted.fit(X_smote, Y_smote)

c:\Users\Alexandre\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(20, 50), learning_rate='adaptive',
              validation_fraction=0.2)

In [517]:
pred = mlp.predict(X_smote)
mat  = confusion_matrix(Y_smote, pred, normalize='true')
mat

array([[0.26717557, 0.73282443],
       [0.00763359, 0.99236641]])

In [518]:
pred2 = mlp.predict(X_test)
mat  = confusion_matrix(Y_test, pred2, normalize='true')
mat

array([[0.2       , 0.8       ],
       [0.05882353, 0.94117647]])